In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F


n_embd=64

In [2]:
#@ Dataset for training gpt, we are using is tiny shakespares dataset:

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-07-23 15:54:14--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2025-07-23 15:54:14 (29.6 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
#@ Reading the file:
with open('input.txt', 'r', encoding='utf-8') as f:
  text=f.read()

In [4]:
print('length of dataset in characters:', len(text))

length of dataset in characters: 1115394


In [5]:
#@ inspecting:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
#@ Unique characters for the code:

chars=sorted(list(set(text)))
# print(chars)
vocab_size=len(chars)
# print(vocab_size) # unique characters len
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
#@ mapping from characters to integer:
str_int={ch:i for i, ch in enumerate(chars)}
int_str={i:ch for i, ch in enumerate(chars)}

encode=lambda s:[str_int[c] for c in s]
decode=lambda l: ''.join([int_str[i] for i in l])

print(encode('hello, i am prijal!'))
print(decode(encode('hello, i am prijal!')))

[46, 43, 50, 50, 53, 6, 1, 47, 1, 39, 51, 1, 54, 56, 47, 48, 39, 50, 2]
hello, i am prijal!


In [8]:
#@ Encoding the entire dataset:
import torch
data=torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
# print(data[:1000])

torch.Size([1115394]) torch.int64


In [9]:
#@ Train-test split:
n=int(0.9*len(data)) # first 90% will be training
train_data=data[:n]
val_data=data[n:]

In [10]:
#@ constructing batch:
def get_batch(split):
  data=train_data if split == 'train' else val_data
  ix=torch.randint(len(data)-block_size, (batch_size,))
  x=torch.stack([data[i:i+block_size] for i in ix])
  y=torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y=x.to(device), y.to(device)
  return x, y

In [11]:
#@ for loss calculation:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [12]:
class Head(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.query=nn.Linear(n_embd, head_size, bias=False)
    self.key=nn.Linear(n_embd, head_size, bias=False)
    self.value=nn.Linear(n_embd, head_size, bias=False)

    self.dropout=nn.Dropout(p=0.5)

  def forward(self, x):
    B, T, C=x.shape #batch, sequence len, channel/features
    q=self.query(x) #(B, T, C)
    k=self.key(x)   #(B, T, C)

    wei=q @ k.transpose(-2, -1) # B, T, T

    wei=wei.masked_fill(self.tril[:T, :T]==0, float('inf')) # B, T, T
    wei=F.softmax(wei, dim=-1) #along the tokens
    wei=self.dropout(wei)

    v=self.value(x) #B, T, C
    out= wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
    return out

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj=nn.Linear(n_embd, n_embd)
    self.dropout=nn.Dropout(p=0.5)

  def forward(self, x):
    out=torch.cat([h(x) for h in self.heads], dims=-1)
    out=self.dropout(self.proj(out))
    return out

In [14]:
class FeedForward(nn.Module):
  def __init__(self, n_embd):
    super().__init__()
    self.net=nn.Sequential(
        nn.Linear(n_embd, 4*n_embd),
        nn.ReLU(),
        nn.Linear(4*n_embd, n_embd),
        nn.Dropout(dropout)
    )

  def forward(self, x):
    return self.net(x)

In [15]:
class Block(nn.Module):
  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size= n_embd // n_head
    self.self_attention=MultiHeadedAttention(n_head, head_size)
    self.ff=FeedForward(n_embd)
    self.ln1=nn.LayerNorm(n_embd)
    self.ln2=nn.LayerNorm(n_embd)

  def forward(self, x):
    x= x + self.self_attention(self.ln1(x))
    x= x + self.ff(self.ln2(x))
    return x

In [16]:
class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table=nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table=nn.Embedding(block_size, n_embd)
    self.blocks=nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
    self.ln_f=nn.LayerNorm(n_embd)
    self.lm_head=nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape  # Batch size, sequence length

     # Apply token and position embeddings
    x = self.token_embedding_table(idx) + self.position_embedding_table(torch.arange(T, device=idx.device))
    x = self.blocks(x)  # Transformer blocks
    x = self.ln_f(x)  # LayerNorm
    logits = self.lm_head(x)  # Convert to (B, T, vocab_size)

    loss = None
    if targets is not None:
        logits = logits.view(B * T, vocab_size)  # Reshape for cross-entropy
        targets = targets.view(B * T)
        loss = F.cross_entropy(logits, targets)

    return logits, loss

def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
        logits, _ = self(idx)  # Get logits from the model
        logits = logits[:, -1, :]  # Only consider last time step
        probs = F.softmax(logits, dim=-1)  # Convert to probability distribution
        idx_next = torch.multinomial(probs, num_samples=1)  # Sample next token
        idx = torch.cat((idx, idx_next), dim=1)  # Append to sequence

        # Make sure idx is on the same device as the model
        idx = idx.to(self.token_embedding_table.weight.device)

    return idx


In [17]:
model=BigramLanguageModel()
m=model.to(device)

print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

optimizer=torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iter):
  if iter % eval_interval == 0 or iter == max_iter - 1:
    losses=estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  # sample a batch of data:
  xb, yb = get_batch('train')

  #evaluating the loss function:
  logits, loss= model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

0.209729 M parameters
step 0: train loss 4.3699, val loss 4.3854
step 100: train loss 2.6497, val loss 2.6521
step 200: train loss 2.5100, val loss 2.5242
step 300: train loss 2.4186, val loss 2.4153
step 400: train loss 2.3527, val loss 2.3684
step 500: train loss 2.2970, val loss 2.2995
step 600: train loss 2.2332, val loss 2.2426
step 700: train loss 2.1965, val loss 2.2137
step 800: train loss 2.1512, val loss 2.1743
step 900: train loss 2.1303, val loss 2.1611
step 1000: train loss 2.0896, val loss 2.1224
step 1100: train loss 2.0692, val loss 2.1024
step 1200: train loss 2.0366, val loss 2.0875
step 1300: train loss 2.0216, val loss 2.0794
step 1400: train loss 1.9849, val loss 2.0523
